In [1]:
# import libraries

from pyautocad import Autocad, APoint, distance, aDouble
from math import pi, atan2, degrees, floor, ceil, sqrt
import pandas as pd
import numpy as np
#Find file name "acad_aag.chm - ActiveX and VBA Developer’s Guide"
#Find file name "acadauto.chm - ActiveX and VBA Reference"
#@ Loaction -> "C:\Program Files\Common Files\Autodesk Shared"

In [2]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Drawing1.dwg


In [3]:
doc = acad.ActiveDocument
layers = doc.Layers
new_layer = layers.Add('PV_Layout')
new_layer.color = 2
new_layer = layers.Add('PV_Layout_Adjust')
new_layer.color = 4
new_layer = layers.Add('Skylight')
new_layer.color = 3


In [4]:
# pv = [0,0,0,1,0,0,1,2,0,0,2,0,0,0,0]
# spv = aDouble(pv)
# Panel = acad.model.AddPolyline(spv)

In [5]:
# P2 = Panel.Copy()
# P3 = P2.Move(APoint(0, 0), APoint(2.4,0)) 

# Function to Mark the building

In [6]:
def prepare_dataframe(name_df: str, x: list, y: list) -> pd.DataFrame:
    """
    Prepare dataframe the columes are x, y, name
    
    Args:
        df_nmae (str): name of dataframe
        x (list): x coordinate of mark
        y (list): y coordinate of mark
        
    Returns:
        df_name (pd.DataFrame): prepared dataframe 
    """
    name_df = pd.DataFrame()
    name_df['x'] = x
    name_df['y'] = y

    Name = []
    for N in range(0,len(name_df)):
        Name.append("Point_"+str(N+1))
        
    name_df['Name'] = Name
    
    return name_df


def execute_line_polygon(df: pd.DataFrame,
                         col_x: str='x', 
                         col_y:str ='y')-> object:
    """add polyline polygon from the Mark

    Args:
        df (pd.DataFrame): The Dataframe contains x,y coordinate of the polygon
        col_x (str): The column name of x coordinate to plot. Defaults to 'x'.
        col_y (str): The column name of y coordinate to plot. Defaults to 'y'.
        
    Returns:
        polygond (object): The object of the polygon in AutoCAD
    """
    plot = []
    for poly in range(0,len(df)):
        plot.append(df[col_x][poly])
        plot.append(df[col_y][poly])
        plot.append(0)

        
    plot.append(df[col_x][0])
    plot.append(df[col_y][0])
    plot.append(0) 

    polygon = aDouble(plot)

    #Execute Line 
    polygond = acad.model.AddPolyline(polygon)
    
    return polygond

# Function to Rectangularize the Building

In [7]:
def rotate_polygon(df:pd.DataFrame, polygond:object, ref_point1: int, ref_point2: int)-> object:
    """Rotate polygon as per Marker
    
    Args:
        Mark (pd.DataFrame): The Dataframe contains x,y coordinate of the polygon
        polygond (object): The object of polygon in AutoCAD
        
    Returns:
        polygond (object): The object of rotated polygon in AutoCAD
    """

    #Input Reference to Rotate make SN axis
    
    # TODO: Ref_1 and Ref_2 as input
    # Ref_1 = 'Point_7' 
    # Ref_2 = 'Point_8'
    xaxis = df.x[ref_point1 - 1] - df.x[ref_point2 - 1]
    yaxis = df.y[ref_point1 - 1] - df.y[ref_point2 - 1]
    degree = degrees(atan2(yaxis,xaxis))
    degree = -degree

    #Rotate Building as per Marker
    x_ref = df.x[ref_point2 - 1]
    y_ref = df.y[ref_point2 - 1]
    
    Point =  APoint(df.x[ref_point2 - 1],df.y[ref_point2 - 1])
    rotated_polygond = polygond.Rotate(Point,pi*degree/180)
    
    return rotated_polygond, x_ref, y_ref


def rotate_light_polygon(df:pd.DataFrame, polygond:object, x_ref, y_ref)-> object:
    """Rotate polygon as per Marker
    
    Args:
        Mark (pd.DataFrame): The Dataframe contains x,y coordinate of the polygon
        polygond (object): The object of polygon in AutoCAD
        
    Returns:
        polygond (object): The object of rotated polygon in AutoCAD
    """

    #Input Reference to Rotate make SN axis
    xaxis = df.x[2] - df.x[3]
    yaxis = df.y[2] - df.y[3]
    degree = degrees(atan2(yaxis,xaxis))
    degree = -degree

    #Rotate Building as per Marker
    Point =  APoint(x_ref, y_ref)
    rotated_polygond = polygond.Rotate(Point,pi*degree/180)
    
    return rotated_polygond


def record_coordinate_of_polygon(Mark)-> pd.DataFrame:
    """Record coordinate of polygon

    Args:
        Mark: The Dataframe contains x,y coordinate of the polygon
        
    Returns:
        After_Rotate (pd.DataFrame): The Dataframe contains x,y coordinate of the rotated polygon 
    """
    
    x_rot = []
    y_rot = []
    num_point = len(Mark)
    for item in acad.iter_objects("AcDb2dPolyline"):
        for n in range(0,num_point):
            cor = list(item.Coordinate[n])
            x_rot.append(cor[0])
            y_rot.append(cor[1])
            
    After_Rotate = prepare_dataframe('After_Rotate', x_rot, y_rot)
    
    return After_Rotate


def calculate_axis_of_each_point(df: pd.DataFrame, 
                                 col_x: str= 'x',
                                 col_y: str= 'y')-> pd.DataFrame:
    """Calculate Axis of each point (In Advance building design may be used)

    Args:
        df (pd.DataFrame): The DataFrame of polygon

    Returns:
        df (pd.DataFrame): The DataFrame of polygon with degree of each point
    """
    
    deg = []
    for d in range(1,len(df)):
        xaxis = df[col_x][d] - df[col_x][d-1]
        yaxis = df[col_y][d] - df[col_y][d-1]
        degree = abs(degrees(atan2(yaxis,xaxis)))
        deg.append(degree)
        
    xaxis = df[col_x][0] - df[col_x][len(df)-1]
    yaxis = df[col_y][0] - df[col_y][len(df)-1]
    degree = abs(degrees(atan2(yaxis,xaxis)))
    deg.append(degree)
    df['deg'] = deg
    
    return df


def calculate_difference_length_of_each_point(df: pd.DataFrame, 
                                              col_x: str= 'x',
                                              col_y: str= 'y')-> pd.DataFrame:
    """Calculate Difference Legnth of each point 

    Args:
        df (pd.DataFrame): The DataFrame of polygon

    Returns:
        df (pd.DataFrame): The DataFrame of polygon with difference length of each point
    """
    d_x = []
    d_y = []
    
    for d in range(0,len(df)-1):
        x_diff = abs(df[col_x][d+1]-df[col_x][d])
        y_diff = abs(df[col_y][d+1]-df[col_y][d])
        d_x.append(x_diff)
        d_y.append(y_diff)
        

    x_diff = abs(df[col_x][0]-df[col_x][len(df)-1])
    y_diff = abs(df[col_y][0]-df[col_y][len(df)-1])
    d_x.append(x_diff)
    d_y.append(y_diff)
    df["d_x"] = d_x
    df["d_y"] = d_y
    df["length"] = np.sqrt(df["d_x"]**2 + df["d_y"]**2)
    
    return df


def rectangularized_the_building(df: pd.DataFrame, 
                                 col_x: str= 'x',
                                 col_y: str= 'y')-> pd.DataFrame:
    """Rectangularized the building

    Args:
        df (pd.DataFrame): The DataFrame of polygon

    Returns:
        df (pd.DataFrame): The DataFrame of polygon with x,y coordinate of each point properly
    """
    #To adjust building coordinate to be should be building sizing 
    # (Have to add-on f(x) of not reatangular building)
    Nx = []
    Ny = []
    Nx.append(df[col_x][0])
    Ny.append(df[col_y][0])
    for a in range(0,len(df)-2):
        if df['d_x'][a] < df['d_y'][a]:
            Nx.append(df[col_x][a])
            Ny.append(df[col_y][a+1])
        else:
            Nx.append(df[col_x][a+1])
            Ny.append(df[col_y][a])

    if df['d_x'][len(df)-1] < df['d_y'][len(df)-1]:
        Nx.append(df[col_x][0])
        Ny.append(df[col_y][len(df)-1])
    else:
        Nx.append(df[col_x][len(df)-2])
        Ny.append(df[col_y][0])
        
    Proper_Polygon = prepare_dataframe('Proper_Polygon', Nx, Ny)
    
    return Proper_Polygon


def prepare_dataframe_of_proper_building(df:pd.DataFrame)-> pd.DataFrame:
    """Prepare dataframe of proper building

    Args:
        df (pd.DataFrame): The DataFrame of rotated polygon

    Returns:
        df (pd.DataFrame): The DataFrame of rotated polygon with additional columns
    """
    
    df = calculate_axis_of_each_point(df)
    df = calculate_difference_length_of_each_point(df)
    df = rectangularized_the_building(df)
    
    return df


def find_main_walkway(Proper_Polygon: pd.DataFrame)-> None:
    calculate_difference_length_of_each_point(Proper_Polygon)
    max_length = Proper_Polygon['length'].max()

    for i in range(len(Proper_Polygon)):
        if Proper_Polygon['length'][i] == max_length:
            index = i
            break

    if index != len(Proper_Polygon)-1:
        index_min = index - 1
        index_max = index
    else:
        index_min = 0
        index_max = index

    if Proper_Polygon['x'][index_min] == Proper_Polygon['x'][index_max]:
        print('Main Walkway along Y-axis')
    elif Proper_Polygon['y'][index_min] == Proper_Polygon['y'][index_max]:
        print('Main Walkway along X-axis')


def delete_polygon()-> object:
      """Deleted Prelimary Input 

      Args:
          None: The object of polygon in AutoCAD

      Returns:
          None: The object of polygon in AutoCAD that has been deleted
      """
      for item in acad.iter_objects():
            item.Delete()

In [8]:
def generate_empty_lists(num_lists: int, name_list: str)-> dict:
    empty_lists = {}
    for i in range(num_lists):
        list_name = f"{name_list}_{i+1}_list"
        empty_lists[list_name] = []
    return empty_lists



In [9]:
# # Generate empty list to ask user input for the mark
# x = []
# y = []
# num_vertices = int(input("Enter number of vertices: "))
# for i in range(num_vertices):
#     x.append(float(input(f"Enter x{i+1}: ")))
#     y.append(float(input(f"Enter y{i+1}: ")))

In [10]:
# # Generate Input Coordinate of skyight

# num_lists_to_generate = int(input("Enter number of skylight lists to generate: "))
# result = generate_empty_lists(num_lists_to_generate, 'skylight')
# for key in result:
#     for i in range(4):
#         result[key].append(float(input(f"Enter x{i+1}: ")))
#         result[key].append(float(input(f"Enter y{i+1}: ")))
#         result[key].append(0)

In [11]:
# Input Coordinate 
# Format 
# x = [x1,x2,x3,x4,x5,x6,x7,x8]
# y = [y1,y2,y3,y4,y5,y6,y7,y8]

x = [642879.55,
     642903.56,
     642903.90,
     642905.74,
     642905.47,
     642907.10,
     642906.61,
     642880.00]

y = [1507732.21,
     1507733.77,
     1507691.52,
     1507691.64,
     1507681.24,
     1507680.92,
     1507670.18,
     1507670.04]

ref_point1 = 7
ref_point2 = 8

# Input Coordinate of skyight (right area)
# x = [x1,x2,x3,x4,x5,x6,x7,x8]
# y = [y1,y2,y3,y4,y5,y6,y7,y8]

sx1 = [642895.07, 642901.27, 642901.27, 642895.07]
sy1 = [1507723.66, 1507723.66, 1507722.66, 1507722.66]

sx2 = [642895.07, 642901.27, 642901.27, 642895.07]
sy2 = [1507719.38, 1507719.38, 1507718.38, 1507718.38]

sx3 = [642895.07, 642901.27, 642901.27, 642895.07]
sy3 = [1507714.85, 1507714.85, 1507713.85, 1507713.85]

sx4 = [642895.07, 642901.27, 642901.27, 642895.07]
sy4 = [1507709.54, 1507709.54, 1507708.54, 1507708.54]

sx5 = [642895.07, 642901.27, 642901.27, 642895.07]
sy5 = [1507705.00, 1507705.00, 1507704.00, 1507704.00]

sx6 = [642895.07, 642901.27, 642901.27, 642895.07]
sy6 = [1507699.69, 1507699.69, 1507698.69, 1507698.69]

sx7 = [642895.07, 642901.27, 642901.27, 642895.07]
sy7 = [1507694.38, 1507694.38, 1507693.38, 1507693.38]

sx8 = [642895.07, 642901.27, 642901.27, 642895.07]
sy8 = [1507689.07, 1507689.07, 1507688.07, 1507688.07]

# Input Coordinate of skyight (left area)
# x = [x1,x2,x3,x4,x5,x6,x7,x8]
# y = [y1,y2,y3,y4,y5,y6,y7,y8]
sx9 = [642887.18, 642893.38, 642893.38, 642887.18]
sy9 = [1507723.66, 1507723.66, 1507722.66, 1507722.66]

sx10 = [642887.18, 642893.38, 642893.38, 642887.18]
sy10 = [1507719.38, 1507719.38, 1507718.38, 1507718.38]
# # Input coordinate of ventilation
# vx1 = [642885.44, 642887.17, 642887.17, 642885.44] 
# vy1 = [1507724.50, 1507724.50, 1507723.66, 1507723.66]

In [12]:
# Polygon of Mark
Mark = prepare_dataframe('Mark', x, y)
polygond = execute_line_polygon(Mark)
polygond, x_ref, y_ref = rotate_polygon(Mark, polygond, ref_point1, ref_point2)
After_Rotate = record_coordinate_of_polygon(Mark)
Proper_Polygon = prepare_dataframe_of_proper_building(After_Rotate)
find_main_walkway(Proper_Polygon)
delete_polygon()

Main Walkway along Y-axis


In [13]:
# Polygon of sky light
skylight_1 = prepare_dataframe('skylight_1', sx1, sy1)
polygond = execute_line_polygon(skylight_1)
polygond = rotate_light_polygon(skylight_1, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_1)
skylight_1 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_2 = prepare_dataframe('skylight_2', sx2, sy2)
polygond = execute_line_polygon(skylight_2)
polygond = rotate_light_polygon(skylight_2, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_2)
skylight_2 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_3 = prepare_dataframe('skylightt_3', sx3, sy3)
polygond = execute_line_polygon(skylight_3)
polygond = rotate_light_polygon(skylight_3, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_3)
skylight_3 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_4 = prepare_dataframe('skylight_4', sx4, sy4)
polygond = execute_line_polygon(skylight_4)
polygond = rotate_light_polygon(skylight_4, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_4)
skylight_4 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_5 = prepare_dataframe('skylight_5', sx5, sy5)
polygond = execute_line_polygon(skylight_5)
polygond = rotate_light_polygon(skylight_5, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_5)
skylight_5 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_6 = prepare_dataframe('skylight_6', sx6, sy6)
polygond = execute_line_polygon(skylight_6)
polygond = rotate_light_polygon(skylight_6, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_6)
skylight_6 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_7 = prepare_dataframe('skylight_7', sx7, sy7)
polygond = execute_line_polygon(skylight_7)
polygond = rotate_light_polygon(skylight_7, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_7)
skylight_7 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_8 = prepare_dataframe('skylight_8', sx8, sy8)
polygond = execute_line_polygon(skylight_8)
polygond = rotate_light_polygon(skylight_8, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_8)
skylight_8 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_9 = prepare_dataframe('skylight_9', sx9, sy9)
polygond = execute_line_polygon(skylight_9)
polygond = rotate_light_polygon(skylight_9, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_9)
skylight_9 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

skylight_10 = prepare_dataframe('skylight_10', sx10, sy10)
polygond = execute_line_polygon(skylight_10)
polygond = rotate_light_polygon(skylight_10, polygond, x_ref, y_ref)
After_Rotate = record_coordinate_of_polygon(skylight_10)
skylight_10 = prepare_dataframe_of_proper_building(After_Rotate)
delete_polygon()

# ventilation_1 = prepare_dataframe('skylight_8', vx1, vy1)
# polygond = execute_line_polygon(ventilation_1)
# polygond = rotate_light_polygon(ventilation_1, polygond)
# After_Rotate = record_coordinate_of_polygon(ventilation_1)
# ventilation_1 = prepare_dataframe_of_proper_building(After_Rotate)
# delete_polygon()

In [14]:
# create list of skylight
# Format
# skylight_1_list = [x1,y1,0,x2,y2,0,x3,y3,0,x4,y4,0] and so on t
# to deal with avioding skylight when zone area division

skylight_1_list = []
skylight_2_list = []
skylight_3_list = []
skylight_4_list = []
skylight_5_list = []
skylight_6_list = []
skylight_7_list = []
skylight_8_list = []
skylight_9_list = []
skylight_10_list = []
# ventilation_1_list = []

for i in range(4):
    skylight_1_list.append(skylight_1.x[i])
    skylight_1_list.append(skylight_1.y[i])
    skylight_1_list.append(0)
    
    skylight_2_list.append(skylight_2.x[i])
    skylight_2_list.append(skylight_2.y[i])
    skylight_2_list.append(0)
    
    skylight_3_list.append(skylight_3.x[i])
    skylight_3_list.append(skylight_3.y[i])
    skylight_3_list.append(0)
    
    skylight_4_list.append(skylight_4.x[i])
    skylight_4_list.append(skylight_4.y[i])
    skylight_4_list.append(0)

    skylight_5_list.append(skylight_5.x[i])
    skylight_5_list.append(skylight_5.y[i])
    skylight_5_list.append(0)
    
    skylight_6_list.append(skylight_6.x[i])
    skylight_6_list.append(skylight_6.y[i])
    skylight_6_list.append(0)
    
    skylight_7_list.append(skylight_7.x[i])
    skylight_7_list.append(skylight_7.y[i])
    skylight_7_list.append(0)
    
    skylight_8_list.append(skylight_8.x[i])
    skylight_8_list.append(skylight_8.y[i])
    skylight_8_list.append(0)
    
    skylight_9_list.append(skylight_9.x[i])
    skylight_9_list.append(skylight_9.y[i])
    skylight_9_list.append(0)
    
    skylight_10_list.append(skylight_10.x[i])
    skylight_10_list.append(skylight_10.y[i])
    skylight_10_list.append(0)
    # ventilation_1_list.append(vx1[i])
    # ventilation_1_list.append(vy1[i])
    # ventilation_1_list.append(0)

In [15]:
# create list of all skylight 
skylight_list = [skylight_1_list, skylight_2_list, skylight_3_list, skylight_4_list,
                 skylight_5_list, skylight_6_list, skylight_7_list, skylight_8_list,
                 skylight_9_list, skylight_10_list
                #  ventilation_1_list
                 ]

In [16]:
polygond = execute_line_polygon(Proper_Polygon, 'x', 'y')

In [17]:
doc.ActiveLayer = doc.Layers('PV_Layout')
polygond = execute_line_polygon(skylight_1, 'x', 'y')
polygond = execute_line_polygon(skylight_2, 'x', 'y')
polygond = execute_line_polygon(skylight_3, 'x', 'y')
polygond = execute_line_polygon(skylight_4, 'x', 'y')
polygond = execute_line_polygon(skylight_5, 'x', 'y')
polygond = execute_line_polygon(skylight_6, 'x', 'y')
polygond = execute_line_polygon(skylight_7, 'x', 'y')
polygond = execute_line_polygon(skylight_8, 'x', 'y')
polygond = execute_line_polygon(skylight_9, 'x', 'y')
polygond = execute_line_polygon(skylight_10, 'x', 'y')
# polygond = execute_line_polygon(ventilation_1, 'x', 'y')

In [18]:
# create dataframe of all skylight
df_skylight = pd.concat([skylight_1,
                        skylight_2,
                        skylight_3,
                        skylight_4,
                        skylight_5,
                        skylight_6,
                        skylight_7,
                        skylight_8,
                        skylight_9,
                        skylight_10
                        # ventilation_1
                        ], ignore_index=True)

In [19]:
# connect Proper_Polygon and light_polygon into one datdframe
df_building = pd.concat([Proper_Polygon], ignore_index=True)

In [20]:
point1 = APoint(0, 0)
point2 = APoint(642880.00, 1507670.04)
line = acad.model.AddLine(point1, point2)

# Area Zone Division by Y-Axis

In [21]:
doc.ActiveLayer = doc.Layers('0')

In [22]:
# create empty list to store list of 3-tuples of each zone
zone_list = []
# create empty list to store length of each zone
zone_length = []
    
# Step1: list all y of the polygon, then let the lowest y be the bottom of the rectangle, 
all_y = list(df_building['y'])
y_sort = list(set(all_y))
y_sort = sorted(y_sort)

# create empty list to store x of the previous zone prepare to analyze the next zone
x_previous = []

# loop for each y_base
for i in range (len(y_sort)-1): # len(y_sort)-1 because the highest y is the top of the building
    y_base = y_sort[i] # y_base is the lowest y 
    y_top = y_sort[i+1] # y_top is the next y 
    
# Step2: find x corresponding to that y_base
    # include x_previous to analyze new x in the next zone
    if len(x_previous) == 0: # at the first zone, x_previous is empty
        x_sample = list(df_building['x'][df_building['y'] == y_base])
            
    elif len(x_previous) != 0: # at the next zone, x_previous is not empty
        x_sample = list(df_building['x'][df_building['y'] == y_base])
        x_sample.extend(x_previous)
     
    x_previous = [] # reset x_previous to use only previous, not before that       
# x_base_analyze
    x_sample.sort()
    
    # delete repeated x 
    x_analyze = []
    for x in x_sample:
        if x not in x_analyze:
            x_analyze.append(x)
    
    # x_base is the x that has y corresponding which is higher than y_base 
    x_base_list = []
    for x in x_analyze:
        x_col = df_building['x'] == x
        y_col = df_building['y'] > y_base
        df = df_building.loc[x_col & y_col]
        x_base_list.append(df)
        df_analyze = pd.concat(x_base_list)
    x_base = list(df_analyze['x']) 
    x_base.sort()
                    
    # delete repeated x
    x_base_filter = []
    for x in x_base:
        if x not in x_base_filter:
            x_base_filter.append(x)

    # delete x for different skylight
    for x in x_base_filter:
        for ii in range(len(skylight_list)):
            y_skylight_base = skylight_list[ii][1]
            y_skylight_top = skylight_list[ii][4]
            
            if y_skylight_top == y_base:
                x_skylight_left = skylight_list[ii][0]
                x_skylight_right = skylight_list[ii][3]
                if x_skylight_left in x_base_filter:
                    x_base_filter.remove(x_skylight_left)
                if x_skylight_right in x_base_filter:
                    x_base_filter.remove(x_skylight_right)
    
    x_base = x_base_filter
    
    # Step3: record zone and zone_length
    zone_record = []
    zone_length_record = []
    
    for j in range(len(x_base)-1):
        left_x = x_base[j]  # two consecutive x
        right_x = x_base[j+1]
        # record only 
        # x_vertex <-> x_skylight (not x_skylight <-> x_skylight)
        if (left_x in df_skylight.x.values) and (right_x in df_skylight.x.values):
            continue
        z = [left_x, y_base, 0, right_x, y_base, 0, right_x, y_top, 0, left_x, y_top, 0]
        zone_record.append(z)
        
        length = right_x - left_x
        zone_length_record.append(length)

         # draw each zone to check
        polygon = aDouble(z)
        polygond = acad.model.AddPolyline(polygon)
        
        # append x in this zone to consider in the next zone
        if left_x not in x_previous:
            x_previous.append(left_x)
        if right_x not in x_previous:
            x_previous.append(right_x)

    zone_list.append(zone_record)
    zone_length.append(zone_length_record)
    
    print(f"zone {i+1} is done")
    

zone 1 is done
zone 2 is done
zone 3 is done


In [23]:
zone_length

[[27.279776527895592], [25.97619441896677], [24.01787503552623]]

In [24]:
zone_list

[[[642879.8770892619,
   1507670.04,
   0,
   642907.1568657898,
   1507670.04,
   0,
   642907.1568657898,
   1507681.105844595,
   0,
   642879.8770892619,
   1507681.105844595,
   0]],
 [[642879.8770892619,
   1507681.105844595,
   0,
   642905.8532836809,
   1507681.105844595,
   0,
   642905.8532836809,
   1507691.3939622608,
   0,
   642879.8770892619,
   1507691.3939622608,
   0]],
 [[642879.8770892619,
   1507691.3939622608,
   0,
   642903.8949642974,
   1507691.3939622608,
   0,
   642903.8949642974,
   1507732.2115070827,
   0,
   642879.8770892619,
   1507732.2115070827,
   0]]]

# Helpful Funtion for Zone Combination

In [25]:
def is_overlap_with(interval1: list, interval2: list):
    """Express whether two intevals are overlaping
    Each Interval is in format: [x1,x2]
    
    Args:
        interval1 (list): list of two x coordinate of the first interval in format [x1,x2]
        interval2 (list): list of two x coordinate of the second interval in format [x1,x2]
    
    Returns:
        bool: True if two intervals are overlaping, False if not
    
    """
    
    start1, end1 = interval1
    start2, end2 = interval2

    if end1 <= start2:
        return False

    if start2 < end1 < end2:
        return True

    if start2 <= start1 <= end1 <= end2:
        return True

    if start1 < end2 < end1:
        return True

    if end2 <= start1:
        return False


def is_interval_within(interval1, interval2):
    """Check whether interval1 is within interval2

    Args:
        interval1 (list): list of two x coordinate of the first interval in format [x1,x2]
        interval2 (list): list of two x coordinate of the second interval in format [x1,x2]

    Returns:
        bool: True if two intervals are intercept, False if not
    """
    
    start1, end1 = interval1
    start2, end2 = interval2

    if start2 <= start1 <= end1 <= end2:
        return True
    else:
        return False


def filter_intercept_zone(zone_list):
    """filter intercept zone from zone_list

    Args:
        zone_list (list of list): 

    Returns:
        intercept_zone_list (list of list): 
    """
    
    intercept_zone_list = []
    for i in range(len(zone_list) - 1):
        for j in range(i + 1, len(zone_list)):
            zone1 = zone_list[i]
            zone2 = zone_list[j]

            x_interval1 = (zone1[0], zone1[3])
            y_interval1 = (zone1[1], zone1[7])
            x1_range = zone1[3] - zone1[0]
            y1_range = zone1[7] - zone1[1]

            x_interval2 = (zone2[0], zone2[3])
            y_interval2 = (zone2[1], zone2[7])
            x2_range = zone2[3] - zone2[0]
            y2_range = zone2[7] - zone2[1]

            if x1_range <= x2_range:
                short_x_interval = x_interval1
                long_x_interval = x_interval2

            if x1_range > x2_range:
                short_x_interval = x_interval2
                long_x_interval = x_interval1

            if y1_range <= y2_range:
                short_y_interval = y_interval1
                long_y_interval = y_interval2

            if y1_range > y2_range:
                short_y_interval = y_interval2
                long_y_interval = y_interval1

            if is_interval_within(short_x_interval, long_x_interval):
                if is_interval_within(short_y_interval, long_y_interval):
                    if zone1 not in intercept_zone_list:
                        intercept_zone_list.append(zone1)
                    if zone2 not in intercept_zone_list:
                        intercept_zone_list.append(zone2)

    return intercept_zone_list


def remove_biggest_intercept_zone(intercept_zone):
    intercept_zone_copy = intercept_zone.copy()
    x_max = 0
    for zone in intercept_zone_copy:
        x_range = zone[3] - zone[0]
        if x_range > x_max:
            x_max = x_range

    for zone in intercept_zone_copy:
        if zone[3] - zone[0] == x_max:
            zone_to_delete = zone
    intercept_zone_copy.remove(zone_to_delete)

    return intercept_zone_copy


def return_remainder_zone(intercept_zone):
    # Find long_interval (x), short_interval_list, and y_interval
    x_length_max = 0
    for i in range(len(intercept_zone)):
        zone = intercept_zone[i]
        x_length = zone[3] - zone[0]
        if x_length > x_length_max:
            x_length_max = x_length

    for i in range(len(intercept_zone)):
        zone = intercept_zone[i]
        if zone[3] - zone[0] == x_length_max:
            long_interval = [zone[0], zone[3]]
            y_interval = [zone[1], zone[7]]
            zone_to_delete = zone
    intercept_zone.remove(zone_to_delete)

    short_interval_list = []
    for i in range(len(intercept_zone)):
        zone = intercept_zone[i]
        short_interval = [zone[0], zone[3]]
        short_interval_list.append(short_interval)

    # Find x_interval intercept of long_interval and short_interval
    x_answer = []
    for short_interval in short_interval_list:
        x_interval = []
        min_long = min(long_interval)
        max_long = max(long_interval)
        min_short = min(short_interval)
        max_short = max(short_interval)
        if max_short <= min_long:
            print("no intercept")

        elif (min_short <= min_long) and (min_long <= max_short) and (max_short <= max_long):
            x_each_interval = [max_short, max_long]
            x_interval.append(x_each_interval)


        elif (min_long <= min_short) and (max_short < max_long):
            x_each_interval1 = [min_long, min_short]
            x_each_interval2 = [max_short, max_long]
            x_interval.append(x_each_interval1)
            x_interval.append(x_each_interval2)

        elif (min_long <= min_short) and (min_short <= max_long) and (max_long <= max_short):
            x_each_interval = [min_long, min_short]
            x_interval.append(x_each_interval)

        elif min_long == min_short and max_long == max_short:
            print("same interval")

        long_interval = x_interval[-1]
        if len(x_interval) != 1:
            x_answer.append(x_interval[0])
    if x_interval[-1] not in x_answer:
        x_answer.append(x_interval[-1])

    intercept_zone_list = []
    for x in x_answer:
        xmin = x[0]
        xmax = x[1]
        ymin = y_interval[0]
        ymax = y_interval[1]
        com = [xmin, ymin, 0, xmax, ymin, 0, xmax, ymax, 0, xmin, ymax, 0]
        intercept_zone_list.append(com)

    return intercept_zone_list


def take_only_zone_to_analyze(zone_list):
    zone_to_analyze = []
    max_y = 0
    for zone in zone_list:
        highest_y = zone[7]
        if highest_y > max_y:
            max_y = highest_y
    for zone in zone_list:
        if zone[7] == max_y:
            zone_to_analyze.append(zone)

    return zone_to_analyze


def return_zone_length(zone_list):
    zone_length = []
    for zone in zone_list:
        length = zone[3] - zone[0]
        zone_length.append(length)

    return zone_length


def return_zone_length_list(new_zone_list):
    zone_length_list = []
    for zone in new_zone_list:
        zone_length = []
        for z in zone:
            length = z[3] - z[0]
            zone_length.append(length)
        zone_length_list.append(zone_length)

    return zone_length_list


def arrange_zone(combine_zone_all):
    zone_list = []
    for l in combine_zone_all:
        for z in l:
            zone_list.append(z)

    sorted_list = sorted(zone_list, key=lambda x: x[1])

    sorted_y_zone_list = [[sorted_list[0]]]
    for i in range(1, len(sorted_list)):
        current_height = sorted_list[i][1]
        previous_height = sorted_list[i - 1][1]
        if current_height == previous_height:
            sorted_y_zone_list[-1].append(sorted_list[i])
        else:
            sorted_y_zone_list.append([])
            sorted_y_zone_list[-1].append(sorted_list[i])

    new_zone_list = []
    for l in sorted_y_zone_list:
        new_l = sorted(l, key=lambda x: x[0])
        new_zone_list.append(new_l)

    return new_zone_list


def return_status_to_continue_iteration(combine_zone_all):
    """Return statistics to continue iteration of combining zone or not

    Args:
        combine_zone_all (list of list): list of list of zone after combining at each iteration

    Returns:
        bool: True if the iteration should be continued, False if not
    """

    # create dictionary of zone after combining at each iteration
    # key : number of each zone
    # value : (minimum of y of each zone, [maximum y of each zone])
    combine_zone_dict = {}
    for i in range(len(combine_zone_all)):
        combine_zone_dict[i] = min([z[1] for z in combine_zone_all[i]]), [z[7] for z in combine_zone_all[i]]

    records = []  
    for i in range(len(combine_zone_dict) - 1):

        for h in combine_zone_dict[i][1]:
            if h != combine_zone_dict[i + 1][0]:
                records.append(0)
            else:
                records.append(1)

    # if sum of records is not equal to 0, continue iteration (there exist consecutive zone)
    if sum(records) != 0:
        _continue_iteration = True

    else:
        _continue_iteration = False

    return _continue_iteration

# Calculate Area after Combination

In [26]:
zone_list = arrange_zone(zone_list)
for zone in zone_list:
    print(zone)

[[642879.8770892619, 1507670.04, 0, 642907.1568657898, 1507670.04, 0, 642907.1568657898, 1507681.105844595, 0, 642879.8770892619, 1507681.105844595, 0]]
[[642879.8770892619, 1507681.105844595, 0, 642905.8532836809, 1507681.105844595, 0, 642905.8532836809, 1507691.3939622608, 0, 642879.8770892619, 1507691.3939622608, 0]]
[[642879.8770892619, 1507691.3939622608, 0, 642903.8949642974, 1507691.3939622608, 0, 642903.8949642974, 1507732.2115070827, 0, 642879.8770892619, 1507732.2115070827, 0]]


In [27]:
zone_list_all = []
for zone in zone_list:
    zone_list_all.extend(zone)

for zone in zone_list_all:
    print(zone)

[642879.8770892619, 1507670.04, 0, 642907.1568657898, 1507670.04, 0, 642907.1568657898, 1507681.105844595, 0, 642879.8770892619, 1507681.105844595, 0]
[642879.8770892619, 1507681.105844595, 0, 642905.8532836809, 1507681.105844595, 0, 642905.8532836809, 1507691.3939622608, 0, 642879.8770892619, 1507691.3939622608, 0]
[642879.8770892619, 1507691.3939622608, 0, 642903.8949642974, 1507691.3939622608, 0, 642903.8949642974, 1507732.2115070827, 0, 642879.8770892619, 1507732.2115070827, 0]


In [28]:
# Calculate Area After Combine
x_zone_all = [] # record x-axis length of each zone
y_zone_all = [] # record y-axis length of each zone
area_zone_all = [] # record area of each zone

for i in range(len(zone_list)):
    x_zone = []
    y_zone = []
    area_zone = []
    for j in range(len(zone_list[i])):
        x_axis_length = zone_list[i][j][3] - zone_list[i][j][0] 
        y_axis_length = zone_list[i][j][7] - zone_list[i][j][1] 
        area = x_axis_length*y_axis_length
        area_zone.append(area)
        x_zone.append(x_axis_length)
        y_zone.append(y_axis_length)

    x_zone_all.append(x_zone)
    y_zone_all.append(y_zone)
    area_zone_all.append(area_zone)
    
print("areazone")
print(area_zone_all)
print("x_zone")
print(x_zone_all)
print("y_zone")
print(y_zone_all)


areazone
[[301.87376764311074], [267.2461446919617], [980.3506907889675]]
x_zone
[[27.279776527895592], [25.97619441896677], [24.01787503552623]]
y_zone
[[11.06584459496662], [10.288117665797472], [40.81754482188262]]


# Design Solar Layout

In [29]:
# TODO: select the alignment of PVs

In [30]:
num_pv_per_string = 18

In [31]:
# spec pv and length by standards
pv_width = 1134/1000
pv_length = 2278/1000

pv_gap = 15/1000
pv_column_gap = 400/1000

building_edge = 1000/1000

walkway_width = 400/1000
walkway_endclamp = 100/1000
walkway_pv_gap = 100/1000

In [32]:
#Find main axis
x_axis_index = 9999
y_axis_index = 9999
if max(x_zone) > max(y_zone):
    x_axis_index = x_zone.index(max(x_zone))
else:
    y_axis_index = y_zone.index(max(y_zone))
    
x_axis_index,y_axis_index  

(9999, 0)

In [33]:
Building_side = 2
Req_Main_Walk = walkway_endclamp + walkway_width + 3*walkway_endclamp + Building_side*building_edge
Req_Pv_String = pv_width*num_pv_per_string + pv_gap*(num_pv_per_string-1)

In [34]:
Req_Distance = Req_Main_Walk + Req_Pv_String
Req_Distance

23.467

# PV Layout

In [35]:
zone = zone_list_all[-1]
zone

[642879.8770892619,
 1507691.3939622608,
 0,
 642903.8949642974,
 1507691.3939622608,
 0,
 642903.8949642974,
 1507732.2115070827,
 0,
 642879.8770892619,
 1507732.2115070827,
 0]

In [36]:
No_row_set = 2 
y_1st = zone_list_all[-1][7] - building_edge   # Start point
y_nth = zone_list_all[0][1] + building_edge  # Last point

PV_set = (y_1st - y_nth) / ((pv_length*No_row_set + pv_gap*(No_row_set-1))+(walkway_width+2*walkway_pv_gap))

Main_PV_set = floor(PV_set)
Portion_PV_set = (PV_set - Main_PV_set)*(pv_length*No_row_set+pv_gap*(No_row_set-1)+walkway_width+2*walkway_pv_gap)/(pv_length+pv_gap)
Portion_PV_set = floor(Portion_PV_set)

PV_set = No_row_set*Main_PV_set+Portion_PV_set
PV_set

23

In [37]:
#Start from upper

pv = []
#1st Panel

#1st point in 1st panel
# "x" (left) + building edge
x = zone[0] + building_edge 
# "y" (From above) - building edge, walk way, cable tray, life line & water line
y = zone[7] - building_edge 
z = 0
edge = [x,y,z]
pv.extend(edge)

#2nd point in 1st panel
x = x + pv_width
y = y
z = 0
edge = [x,y,z]
pv.extend(edge)

#3rd point in 1st panel
x = x 
y = y - pv_length
z = 0
edge = [x,y,z]
pv.extend(edge)

#4th point in 1st panel
x = zone[0] + building_edge 
y = zone[7] - building_edge - pv_length
z = 0
edge = [x,y,z]
pv.extend(edge)

#Cover Rectangular point in 1st panel
x = zone[0] + building_edge  #Back to start point
y = zone[7] - building_edge #Back to start point
z = 0
edge = [x,y,z]
pv.extend(edge)

spv = aDouble(pv)
Panel = acad.model.AddPolyline(spv)

# gap between skylight & PV array

In [38]:
# #P2 = Panel.Copy()
# no_B = 0
# g = 0
# x = zone_list_all[-1][0] + building_edge  # Back to start point
# y = zone_list_all[-1][7] - building_edge  # Back to start point

# # check wheher x,y is in which zone

        
# Round = 0
# for c in range(0,PV_set):
    
#     if (c>0) & (c%No_row_set==0):
#         no_B += 1  
#     if (c>0) & (Round>No_row_set):
#         Round = 1
#     if (c>0) & (Round<No_row_set):
#         g +=1
        
    
#     print("c : "+str(c)+" Round : "+str(Round)+" g : "+str(g))
#     Round += 1
    
    
#     yi = y-(c*pv_length)-(g*pv_gap)-no_B*(2*walkway_pv_gap+walkway_width)    
#     P2 = Panel.Copy()
#     P2.Move(APoint(x,y),
#             APoint(x,yi)) #Plus in "C" from "left" to "Right
    
#     pv_x_range = [x, x+pv_width]
#     pv_y_range = [yi-pv_length, yi]
    
#     for zone in zone_list_all:
#         if is_overlap_with(pv_x_range, [zone[0], zone[3]]) and is_overlap_with(pv_y_range, [zone[1], zone[7]]):
#             x_length = zone[3] - zone[0]
            
#             No_PV = (x_length-Req_Main_Walk+pv_gap)/(pv_width+pv_gap)
#             No_PV = floor(No_PV) 
    
#     for r in range(1,No_PV):
#         xi = x+(r*pv_width)+(r*pv_gap)
#         P2 = Panel.Copy()
#         P2.Move(APoint(x,y),
#             APoint(xi,yi)) #Minus in "r" from "Up" to "Down"
        
#         pv_x_interval = [xi, xi+pv_width]
#         pv_y_interval = [yi-pv_length, yi]
        
#         for skylight in skylight_list:
#             sky_x_interval = [skylight[0], skylight[3]]
#             sky_y_interval = [skylight[7], skylight[1]]
            
            
#             if (is_overlap_with(pv_x_interval, sky_x_interval) or is_overlap_with(sky_x_interval, pv_x_interval))\
#                 and (is_overlap_with(pv_y_interval, sky_y_interval) or is_overlap_with(sky_y_interval, pv_y_interval)):
#                 P2.Delete()
#                 if yi-pv_length < skylight[7]:
                    
#                     # x_sky_length = skylight[3] - skylight[0]
                    
#                     # No_PV = floor(x_sky_length / pv_width)
                    
#                     # for rr in range(No_PV):
                        
#                     #     xii = skylight[0] + (rr*pv_width) + (rr*pv_gap)
#                     #     yii = skylight[7] - 2*walkway_pv_gap - walkway_width                   

#                     #     P2 = Panel.Copy()
#                     #     P2.Move(APoint(x,y),
#                     #         APoint(xii,yii))
                    
                    
#                     # xii = skylight[0] + (rr*pv_width) + (rr*pv_gap)
#                     yii = skylight[7] - walkway_pv_gap 
#                     P2 = Panel.Copy()
#                     P2.Move(APoint(x,y),
#                         APoint(xi,yii))
#                     # rr += 1
                    
#                     break

## Method1 (Copied)

In [39]:
def move_copy_to_layer(copied_object, target_layer_name):
    # Connect to AutoCAD application
    doc = acad.ActiveDocument

    # Get the Layers collection
    layers = doc.Layers

    # Find the layer by name
    target_layer = None
    for layer in layers:
        if layer.Name == target_layer_name:
            target_layer = layer
            break

    if target_layer is not None:
        # Change the layer property of the copied object
        copied_object.Layer = target_layer_name
        # print(f"Object moved to layer '{target_layer_name}' successfully.")
    else:
        pass
        # print(f"Layer '{target_layer_name}' not found.")

# # Example usage:
# # Assuming P2 is a reference to the copied object and '1' is the target layer name
# P2 = ...  # The reference to the copied object
# target_layer_name = '1'
# move_copy_to_layer(P2, target_layer_name)


In [40]:
doc.ActiveLayer = doc.Layers('PV_Layout')

In [41]:
#P2 = Panel.Copy()
no_B = 0
g = 0
x = zone_list_all[-1][0] + building_edge  # Back to start point
y = zone_list_all[-1][7] - building_edge  # Back to start point

# check wheher x,y is in which zone
        
Round = 0

num_pv_list = []

y_min_list = []
for c in range(0,PV_set):
    num_pv_row = []
    
    if (c>0) & (c%No_row_set==0):
        no_B += 1  
    if (c>0) & (Round>No_row_set):
        Round = 1
    if (c>0) & (Round<No_row_set):
        g +=1
        
    
    #print("c : "+str(c)+" Round : "+str(Round)+" g : "+str(g))
    Round += 1    
    
    
    yi = y-(c*pv_length)-(g*pv_gap)-no_B*(2*walkway_pv_gap+walkway_width)
    y_min_list.append(yi)
        
    if c > 0 and c % No_row_set == 0:
        yi = min(y_min_list) 
        y_min_list.append(yi)
         
    P2 = Panel.Copy()
    P2.Move(APoint(x,y),
            APoint(x,yi)) #Plus in "C" from "left" to "Right
    
    pv_x_range = [x, x+pv_width]
    pv_y_range = [yi-pv_length, yi]
    
    for zone in zone_list_all:
        if is_overlap_with(pv_x_range, [zone[0], zone[3]]) and is_overlap_with(pv_y_range, [zone[1], zone[7]]):
            x_length = zone[3] - zone[0]
            
            No_PV = (x_length-Req_Main_Walk+pv_gap)/(pv_width+pv_gap)
            No_PV = floor(No_PV) 
            print(No_PV)
            
            if No_PV == 18:
                move_copy_to_layer(P2, 'PV_Layout')
            elif No_PV != 18:
                move_copy_to_layer(P2, 'PV_Layout_Adjust')
            break
            
    count_PV = 1
    
    pv_object_list = []
    _jump = False 
    
    for r in range(1,No_PV):
        xi = x+(r*pv_width)+(r*pv_gap)
        P2 = Panel.Copy()
        P2.Move(APoint(x,y),
            APoint(xi,yi)) #Minus in "r" from "Up" to "Down"
        count_PV += 1
        
        pv_object_list.append(P2)
        
        if No_PV == 18:
            move_copy_to_layer(P2, 'PV_Layout')
        elif No_PV != 18:
            move_copy_to_layer(P2, 'PV_Layout_Adjust')
                
        pv_x_interval = [xi, xi+pv_width]
        pv_y_interval = [yi-pv_length, yi]
        
        
        for skylight in skylight_list:
            sky_x_interval = [skylight[0], skylight[3]]
            sky_y_interval = [skylight[7], skylight[1]]
            
            
            if (is_overlap_with(pv_x_interval, sky_x_interval) or is_overlap_with(sky_x_interval, pv_x_interval))\
                and (is_overlap_with(pv_y_interval, sky_y_interval) or is_overlap_with(sky_y_interval, pv_y_interval)):
                P2.Delete()
                
                if yi-pv_length < skylight[7]:
                    _jump = True
                    
                    # x_sky_length = skylight[3] - skylight[0]
                    
                    # No_PV = floor(x_sky_length / pv_width)
                    
                    # for rr in range(No_PV):
                        
                    #     xii = skylight[0] + (rr*pv_width) + (rr*pv_gap)
                    #     yii = skylight[7] - 2*walkway_pv_gap - walkway_width                   

                    #     P2 = Panel.Copy()
                    #     P2.Move(APoint(x,y),
                    #         APoint(xii,yii))
                    
                    xi = x + (r*pv_width) + (r*pv_gap) + 2*walkway_pv_gap + walkway_width    
                    # xii = skylight[0] + (rr*pv_width) + (rr*pv_gap)
                    yii = skylight[7] - walkway_pv_gap 
                    P2 = Panel.Copy()
                    # print(type(P2))
                    P2.Move(APoint(x,y),
                        APoint(xi,yii))
                    
                    pv_object_list.append(P2)
                    
                    y_min_list.append(yii)
                    # rr += 1
                    if No_PV == 18:
                        move_copy_to_layer(P2, 'PV_Layout')
                    elif No_PV != 18:
                        move_copy_to_layer(P2, 'PV_Layout_Adjust')
                    
                    break
            
            if _jump == True:
                for pv in pv_object_list:
                    move_copy_to_layer(pv, 'Skylight')
                
        

18
18
18
18


COMError: (-2147352567, 'Exception occurred.', ('Object was erased', 'AutoCAD.Application', 'C:\\Program Files\\Autodesk\\AutoCAD 2022\\HELP\\OLE_ERR.CHM', -2145386420, None))

In [ ]:
# for o in pv_coordinate:
#     move_copy_to_layer(o, 'Skylight')

### Method2: gap between PV array & PV array

In [ ]:
# #P2 = Panel.Copy()
# no_B = 0
# g = 0
# x = zone_list_all[-1][0] + building_edge  # Back to start point
# y = zone_list_all[-1][7] - building_edge  # Back to start point

# # check wheher x,y is in which zone

# Round = 0
# for c in range(0,PV_set):
    
#     if (c>0) & (c%No_row_set==0):
#         no_B += 1  
#     if (c>0) & (Round>No_row_set):
#         Round = 1
#     if (c>0) & (Round<No_row_set):
#         g +=1
        
    
#     print("c : "+str(c)+" Round : "+str(Round)+" g : "+str(g))
#     Round += 1
    

#     yi = y-(c*pv_length)-(g*pv_gap)-no_B*(2*walkway_pv_gap+walkway_width)  
        
#     P2 = Panel.Copy()
#     P2.Move(APoint(x,y),
#             APoint(x,yi)) #Plus in "C" from "left" to "Right
    
#     pv_x_range = [x, x+pv_width]
#     pv_y_range = [yi-pv_length, yi]
    
#     for zone in zone_list_all:
#         if is_overlap_with(pv_x_range, [zone[0], zone[3]]) and is_overlap_with(pv_y_range, [zone[1], zone[7]]):
#             x_length = zone[3] - zone[0]
            
#             No_PV = (x_length-Req_Main_Walk+pv_gap)/(pv_width+pv_gap)
#             No_PV = floor(No_PV) 
    
#     for r in range(1,No_PV):
#         xi = x+(r*pv_width)+(r*pv_gap)
#         P2 = Panel.Copy()
#         P2.Move(APoint(x,y),
#             APoint(xi,yi)) #Minus in "r" from "Up" to "Down"
        
#         pv_x_interval = [xi, xi+pv_width]
#         pv_y_interval = [yi-pv_length, yi]
        
#         for skylight in skylight_list:
#             sky_x_interval = [skylight[0], skylight[3]]
#             sky_y_interval = [skylight[7], skylight[1]]
            
            
#             if (is_overlap_with(pv_x_interval, sky_x_interval) or is_overlap_with(sky_x_interval, pv_x_interval))\
#                 and (is_overlap_with(pv_y_interval, sky_y_interval) or is_overlap_with(sky_y_interval, pv_y_interval)):
#                 P2.Delete()
#                 if yi-pv_length < skylight[7]:
                    
#                     x_sky_length = skylight[3] - skylight[0]
                    
#                     No_PV = floor(x_sky_length / pv_width)
                    
#                     for r in range(No_PV):
                        
#                         yii = skylight[7] - walkway_pv_gap                 

#                         P2 = Panel.Copy()
#                         P2.Move(APoint(x,y),
#                             APoint(xi,yii))
                    
#                     # yii = skylight[7] - walkway_pv_gap 
#                     # P2 = Panel.Copy()
#                     # P2.Move(APoint(x,y),
#                     #     APoint(xi,yii))
                    
#                     break

### Method3: Skip all skylights, then place PVs nearby skylights

In [ ]:
# #Start from upper

# pv = []
# #1st Panel

# #1st point in 1st panel
# # "x" (left) + building edge
# x = zone[0] + building_edge 
# # "y" (From above) - building edge, walk way, cable tray, life line & water line
# y = zone[7] - building_edge 
# z = 0
# edge = [x,y,z]
# pv.extend(edge)

# #2nd point in 1st panel
# x = x + pv_width
# y = y
# z = 0
# edge = [x,y,z]
# pv.extend(edge)

# #3rd point in 1st panel
# x = x 
# y = y - pv_length
# z = 0
# edge = [x,y,z]
# pv.extend(edge)

# #4th point in 1st panel
# x = zone[0] + building_edge 
# y = zone[7] - building_edge - pv_length
# z = 0
# edge = [x,y,z]
# pv.extend(edge)

# #Cover Rectangular point in 1st panel
# x = zone[0] + building_edge  #Back to start point
# y = zone[7] - building_edge #Back to start point
# z = 0
# edge = [x,y,z]
# pv.extend(edge)

# spv = aDouble(pv)
# Panel = acad.model.AddPolyline(spv)

# #P2 = Panel.Copy()
# no_B = 0
# g = 0
# x = zone_list_all[-1][0] + building_edge  # Back to start point
# y = zone_list_all[-1][7] - building_edge  # Back to start point

# # check wheher x,y is in which zone

        
# Round = 0
# for c in range(0,PV_set):
    
#     if (c>0) & (c%No_row_set==0):
#         no_B += 1  
#     if (c>0) & (Round>No_row_set):
#         Round = 1
#     if (c>0) & (Round<No_row_set):
#         g +=1
        
    
#     print("c : "+str(c)+" Round : "+str(Round)+" g : "+str(g))
#     Round += 1
    
    
#     yi = y-(c*pv_length)-(g*pv_gap)-no_B*(2*walkway_pv_gap+walkway_width)    
#     P2 = Panel.Copy()
#     P2.Move(APoint(x,y),
#             APoint(x,yi)) #Plus in "C" from "left" to "Right
    
#     pv_x_range = [x, x+pv_width]
#     pv_y_range = [yi-pv_length, yi]
    
#     for zone in zone_list_all:
#         if is_overlap_with(pv_x_range, [zone[0], zone[3]]) and is_overlap_with(pv_y_range, [zone[1], zone[7]]):
#             x_length = zone[3] - zone[0]
            
#             No_PV = (x_length-Req_Main_Walk+pv_gap)/(pv_width+pv_gap)
#             No_PV = floor(No_PV) 
    
#     for r in range(1,No_PV):
#         xi = x+(r*pv_width)+(r*pv_gap)
#         P2 = Panel.Copy()
#         P2.Move(APoint(x,y),
#             APoint(xi,yi)) #Minus in "r" from "Up" to "Down"
        
#         pv_x_interval = [xi, xi+pv_width]
#         pv_y_interval = [yi-pv_length, yi]
        
#         for skylight in skylight_list:
#             sky_x_interval = [skylight[0], skylight[3]]
#             sky_y_interval = [skylight[7], skylight[1]]
            
            
#             if (is_overlap_with(pv_x_interval, sky_x_interval) or is_overlap_with(sky_x_interval, pv_x_interval))\
#                 and (is_overlap_with(pv_y_interval, sky_y_interval) or is_overlap_with(sky_y_interval, pv_y_interval)):
#                 P2.Delete()
#                 break
            
# for skylight in skylight_list:
#     #Start from upper

#     pv = []
#     #1st Panel

#     #1st point in 1st panel
#     # "x" (left) + building edge
#     x = skylight[0] 
#     # "y" (From above) - building edge, walk way, cable tray, life line & water line
#     y = skylight[7] - 2*walkway_pv_gap - walkway_width
#     z = 0
#     edge = [x,y,z]
#     pv.extend(edge)

#     #2nd point in 1st panel
#     x = x + pv_width
#     y = y
#     z = 0
#     edge = [x,y,z]
#     pv.extend(edge)

#     #3rd point in 1st panel
#     x = x 
#     y = y - pv_length
#     z = 0
#     edge = [x,y,z]
#     pv.extend(edge)

#     #4th point in 1st panel
#     x = skylight[0] 
#     y = skylight[7] - 2*walkway_pv_gap - walkway_width - pv_length
#     z = 0
#     edge = [x,y,z]
#     pv.extend(edge)

#     #Cover Rectangular point in 1st panel
#     x = skylight[0] 
#     y = skylight[7] - 2*walkway_pv_gap - walkway_width #Back to start point
#     z = 0
#     edge = [x,y,z]
#     pv.extend(edge)

#     spv = aDouble(pv)
#     Panel = acad.model.AddPolyline(spv)
    
    
    
#     x_length_skylight = skylight[3] - skylight[0]
#     No_PV = floor(x_length_skylight / pv_width)
    

#     for r in range(1,No_PV):
#         xi = x+(r*pv_width)+(r*pv_gap)
#         P2 = Panel.Copy()
#         P2.Move(APoint(x,y),
#             APoint(xi,y)) #Minus in "r" from "Up" to "Down"

In [ ]:
# import win32com.client
# from pyautocad import Autocad, APoint

# def create_block(acad, objects, block_name, insertion_point):
#     # Create a selection set and add the objects to it
#     selection = acad.ActiveDocument.SelectionSets.Add("TempSelection")
#     for obj in objects:
#         selection.Add(obj)

#     # Create the block definition
#     block = acad.ActiveDocument.Blocks.Add(insertion_point, block_name)

#     # Paste the objects into the block definition
#     block.InsertionPoint = insertion_point
#     block.PasteObjects(selection)

#     # Erase the temporary selection set
#     selection.Delete()

#     return block

# # Connect to AutoCAD
# acad = win32com.client.Dispatch("AutoCAD.Application")

# # Get the active document
# doc = acad.ActiveDocument

# # Get the objects you want to group (for example, select two circles)
# acad.ActiveDocument.Utility.GetEntity()
# object1 = acad.ActiveDocument.Utility.GetObject(0)
# acad.ActiveDocument.Utility.GetEntity()
# object2 = acad.ActiveDocument.Utility.GetObject(0)

# # Choose an insertion point for the block
# insertion_point = APoint(10, 10)

# # Create a block definition containing the two objects
# block_name = "MyGroup"
# block = create_block(acad, [object1, object2], block_name, insertion_point)

# # Insert the block into the drawing
# doc.ModelSpace.InsertBlock(insertion_point, block_name, 1.0, 1.0, 1.0, 0)


In [ ]:
# def create_block(acad, objects, block_name, insertion_point):
#     # Create a selection set and add the objects to it
#     selection = acad.ActiveDocument.SelectionSets.Add("TempSelection")
#     for obj in objects:
#         selection.Add(obj)

#     # Create the block definition
#     block = acad.ActiveDocument.Blocks.Add(insertion_point, block_name)

#     # Paste the objects into the block definition
#     block.InsertionPoint = insertion_point
#     block.PasteObjects(selection)

#     # Erase the temporary selection set
#     selection.Delete()

#     return block

In [ ]:
# # Get the active document
# doc = acad.ActiveDocument

# a = [0,0,0,1,0,0,1,1,0,0,1,0]
# b = [2,0,0,4,0,0,4,2,0,2,2,0]

# ob_a = aDouble(a)
# ob_b = aDouble(b)
# object1 = acad.model.AddPolyline(ob_a)
# object2 = acad.model.AddPolyline(ob_b)

In [ ]:
# ip = APoint(0, 0, 0)


# # adding block to documents block collection
# b1 = acad.doc.Blocks.Add(ip, "Test_block_1")

# # adding geometries to block
# pl = b1.AddPolyline(aDouble(0, 0, 0, 10000, 0, 0, 10000, 5000, 0, 0, 5000, 0, 0, 0, 0))
# l = b1.AddLine(APoint(0, 250, 0), APoint(10000, 250, 0))


# # adding block instance to drawing (creating a reference to the block)
# block_ref1 = acad.model.InsertBlock(APoint(50, 50, 0), "Test_block_1", 1, 1, 1, 0)

In [ ]:
# # Get the active document
# doc = acad.ActiveDocument



# # Get the objects you want to group (for example, select two circles)
# doc = acad.ActiveDocument

# a = [0,0,0,1,0,0,1,1,0,0,1,0]
# b = [2,0,0,4,0,0,4,2,0,2,2,0]

# ob_a = aDouble(a)
# ob_b = aDouble(b)
# object1 = acad.model.AddPolyline(ob_a)
# object2 = acad.model.AddPolyline(ob_b)

# # Choose an insertion point for the block
# insertion_point = APoint(0, 0)

# # Create a block definition containing the two objects
# block_name = "MyGroup"
# block = create_block(acad, [object1, object2], block_name, insertion_point)

# # Insert the block into the drawing
# doc.ModelSpace.InsertBlock(insertion_point, block_name, 1.0, 1.0, 1.0, 0)

# Calculate Inverter

In [ ]:
num_pv_each_column_list 

In [ ]:
num_pv_per_string_list = []
if len(num_pv_each_column_list)%No_column_set == 0:
    for i in range (0,len(num_pv_each_column_list )):
        num_PV = num_pv_each_column_list[i] * No_column_set
        num_pv_per_string_list.append(num_PV)
        
elif len(num_pv_each_column_list )%No_column_set != 0:
    for i in range (0,len(num_pv_each_column_list)-1):
        num_PV = num_pv_each_column_list[i] * No_column_set
        num_pv_per_string_list.append(num_PV)
    num_PV = num_pv_each_column_list [-1] # last num
    num_pv_per_string_list.append(num_PV)
    num_pv_per_string_list.append(num_PV)

In [ ]:
num_pv_per_string_list

In [ ]:
#Find Suitable WalkWay
#Partial Analyze based on sub-axis



In [ ]:
#Calculate PV per column (Big Area)
# 1 tier is PV per column == Invertor string
# 2 tier is PV per column == (Invertor string)/2
# 3 tier is others



In [ ]:
# #String Number Cal
# No_string = (y_axis_length-Req_Main_Walk)/Req_Pv_String
# No_string

In [ ]:
#number of PV in each column
No_PV = (y_zone[0]-Req_Main_Walk+pv_gap)/(pv_width+pv_gap)
No_PV = floor(No_PV)
No_PV                

In [ ]:
Building_side = 2
Req_Main_Walk = walkway_endclamp + walkway_width + 3*walkway_endclamp + Building_side*building_edge
Req_Pv_String = pv_width*num_pv_per_string + pv_gap*(num_pv_per_string-1)

In [ ]:
Req_Distance = Req_Main_Walk + Req_Pv_String
Req_Distance

In [ ]:
#PV number
No_PV = (y_zone[0]-Req_Main_Walk+pv_gap)/(pv_width+pv_gap)
No_PV = floor(No_PV)
No_PV    

In [ ]:
for y_zone in y_zone_all:
    for y in y_zone:
        No_PV = (y-Req_Main_Walk+pv_gap)/(pv_width+pv_gap)
        No_PV = floor(No_PV)
        print(No_PV)


In [ ]:
y_zone_all

## Create Main Walkway

In [ ]:
#Walk Way create (Main)
#Start from lower
Main_way = []
z = 0
x = combine_zone[0][0] + building_edge  #Start point
y = combine_zone[0][1] + building_edge  #Lower Main Edge
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

x = combine_zone[0][3] - building_edge  
y = y  
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

x = x  
y = y + walkway_width #Upper Main Edge
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

x = combine_zone[0][0] + building_edge  
y = y 
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

x = x  
y = y - walkway_width  
Main_edge = [x,y,z]
Main_way.extend(Main_edge)

mw = aDouble(Main_way)
MainWay = acad.model.AddPolyline(mw)
MainWay.Color = 2

# Create DC Line

In [ ]:
# create empty list to store value of edge of x and y to find contact point of DC cables of each string
x_contact_point_list = []
y_contact_point_list = []
if len(x_edge_list)%2 == 0: # if the number of x_edge_list is even
    for i in range (0,len(x_edge_list),2):
        x1 = x_edge_list[i]
        x2 = x_edge_list[i+1]+pv_length
        y1 = y_edge_list[i]
        y2 = y_edge_list[i+1]
        x_contact_point_list.append(x1)
        x_contact_point_list.append(x2)
        y_contact_point_list.append(y1)
        y_contact_point_list.append(y2)          
        
elif len(x_edge_list)%2 != 0: # if the number of x_edge_list is odd
    for i in range (0,len(x_edge_list)-1,2):
        x1 = x_edge_list[i]
        x2 = x_edge_list[i+1]+pv_length
        y1 = y_edge_list[i]
        y2 = y_edge_list[i+1]
        x_contact_point_list.append(x1)
        x_contact_point_list.append(x2)
        y_contact_point_list.append(y1)
        y_contact_point_list.append(y2)    
    x1 = x_edge_list[-1] # last x
    x2 = x_edge_list[-1]+pv_length
    y1 = y_edge_list[-1] # last y
    y2 = y_edge_list[-1]
    x_contact_point_list.append(x1)
    x_contact_point_list.append(x2)
    y_contact_point_list.append(y1)
    y_contact_point_list.append(y2)

In [ ]:
# input coordinate of inverter
inverter_coor = [642905.8286, 1507675.5392]

### Create DC Cables New Version

In [ ]:
# create dc line from contact point to inverter

y_center_main_way = (Main_way[1]+Main_way[7])/2 # y coordinate of center of main way
# create empty list to store length of dc line of each PV string
dc_length_list = []
for i in range (len(x_contact_point_list)):
    x1 = x_contact_point_list[i]
    y1 = y_contact_point_list[i]
    p1 = APoint(x1,y1)
        
    x2 = x1
    y2 = y_center_main_way
    p2 = APoint(x2,y2)
        
    x3 = inverter_coor[0]
    y3 = y_center_main_way
    p3 = APoint(x3,y3)
        
    x4 = x3
    y4 = inverter_coor[1]
    p4 = APoint(x4,y4)

    if i%2 == 0:
        line1 = acad.model.AddLine(p1,p2)
        line1.Color = 1
        line2 = acad.model.AddLine(p2,p3)
        line2.Color = 1
        line3 = acad.model.AddLine(p3,p4)
        line3.Color = 1
    elif i%2 != 0:
        line1 = acad.model.AddLine(p1,p2)
        line1.Color = 3
        line2 = acad.model.AddLine(p2,p3)
        line2.Color = 3
        line3 = acad.model.AddLine(p3,p4)
        line3.Color = 3
    
    length_part_1 = distance(p1,p2)
    length_part_2 = distance(p2,p3)
    length_part_3 = distance(p3,p4)
    
    dc_length = length_part_1 + length_part_2 + length_part_3 # length of dc line of each PV string
    dc_length_list.append(dc_length)

In [ ]:
dc_length_list

# Voltage Calculation

### Formula
Vdrop (V) = Iwire (A) × Rwire(Ω)
          = Iwire (A) × (2 × L(m) × Rwire (Ω/km) / 1000(m/km))

Vdrop (V) = num_PV x Voltage of each PV (in 1 string)


In [ ]:
# Assume that this project use 4 mm2 DC cable

Iwire = 14.47 #unit: A
Rwire = 5.09
voltage_drop_list = [] # create empty list to store voltage drop of each PV string
for l in dc_length_list:
    r = 2*l*Rwire/1000 #unit: ohm
    voltage_drop = Iwire * r
    voltage_drop_list.append(voltage_drop)

In [ ]:
# create pandas datafrane where colums are voltage drop, length of dc line, and number of PV
DC_cable_dataframe = pd.DataFrame()
DC_cable_dataframe['length (m)'] = dc_length_list
DC_cable_dataframe['num_PV_per_string'] = num_pv_per_string_list
DC_cable_dataframe['voltage_at_start_terminal (V)'] = DC_cable_dataframe['num_PV_per_string'] * 51.02

DC_cable_dataframe['voltage_drop (V)'] = voltage_drop_list
DC_cable_dataframe['percent_voltage_drop (%)'] = DC_cable_dataframe['voltage_drop (V)'] / DC_cable_dataframe['voltage_at_start_terminal (V)'] * 100


for percent in DC_cable_dataframe['percent_voltage_drop (%)']:
    if percent < 3:
        DC_cable_dataframe['DC_cable_size (mm^2)'] = 4
    elif 3 < percent and percent < 6:
        DC_cable_dataframe['DC_cable_size (mm^2)'] = 6
    elif 6 < percent and percent < 9:
        DC_cable_dataframe['DC_cable_size (mm^2)'] = 10

for size in DC_cable_dataframe['DC_cable_size (mm^2)']:
    if size == 4:
        DC_cable_dataframe['DC_cable_price (THB)'] = DC_cable_dataframe['length (m)'] * 20
    elif size == 6:
        DC_cable_dataframe['DC_cable_price (THB)'] = DC_cable_dataframe['length (m)'] * 25
    elif size == 10:
        DC_cable_dataframe['DC_cable_price (THB)'] = DC_cable_dataframe['length (m)'] * 45
        
DC_cable_dataframe


In [ ]:
total_DC_cable_price = DC_cable_dataframe['DC_cable_price (THB)'].sum()
print(f'Total Price is {total_DC_cable_price} Baht')

# Create Branch

In [ ]:
#Walk Way create (Branch)
branch_way = []
x = combine_zone[1][0] + building_edge  #start point
y = combine_zone[1][7] - building_edge - walkway_width - 0*walkway_endclamp #start point
z = 0
edge = [x,y,z]
branch_way.extend(edge)


x = x + walkway_width  #right point
y = y #right point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x #right bottom point
y = combine_zone[1][1] + building_edge + walkway_width   #right bottom point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x - walkway_width #left bottom point
y = y  #left bottom point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = combine_zone[1][0] + building_edge  #start point
y = combine_zone[1][7] - building_edge - walkway_width - 0*walkway_endclamp #start point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

bw = aDouble(branch_way)
BranchWay = acad.model.AddPolyline(bw)
BranchWay.Color = 2

In [ ]:
#Expand branch line

Pre_Walk = No_column_set*pv_length+(No_column_set-1)*pv_gap+walkway_pv_gap
Back_Walk = walkway_pv_gap + walkway_width
x = combine_zone[1][0] + building_edge  #start point
y = combine_zone[1][7] - building_edge - walkway_width - 0*walkway_endclamp #start point
Round = 0
pv = 0
m_point = Pre_Walk*(Round+1) + Round*Back_Walk
BranchWay.Move(APoint(x,y),
            APoint(x+m_point,y)) #Plus in "C" from "left" to "Right"
x = x+m_point

#bw2 = BranchWay.Copy()
if PV_set%No_column_set!=0:
    PV_set_Round = PV_set-1
if PV_set%No_column_set==0:
    PV_set_Round = PV_set
    
while pv < PV_set_Round:
    #print("pv : "+str(pv))
    bw2 = BranchWay.Copy()
    m_point = (2*walkway_pv_gap + walkway_width + No_column_set*pv_length + (No_column_set-1)*pv_gap)*Round
    #m_point = Pre_Walk*(Round+1) + Round*Back_Walk
    pv = pv + No_column_set
    Round += 1
    
    bw2.Move(APoint(x,y),
            APoint(x+m_point,y)) #Plus in "C" from "left" to "Right"
    
    

# PV Layout in 1st Zone

In [ ]:
#Analyzing Length to Plan
x_axis_length = combine_zone[0][3] - combine_zone[0][0]
y_axis_length = combine_zone[0][7] - combine_zone[0][1]
area = x_axis_length*y_axis_length
area_zone.append(area)
x_zone.append(x_axis_length)
y_zone.append(y_axis_length)

In [ ]:
combine_zone[0][3], combine_zone[0][0]

In [ ]:
combine_zone[0][7], combine_zone[0][1]

In [ ]:
combine_zone[0][7], combine_zone[1][1] + building_edge - walkway_endclamp

In [ ]:
Lower_Main_Way = combine_zone[1][1] + building_edge - walkway_endclamp
Upper_Area_Edge = combine_zone[0][7]

if Lower_Main_Way > Upper_Area_Edge:
    PV_Upper_Area_2_Edge = Lower_Main_Way
    
PV_Upper_Area_2_Edge
PV_Lower_Area_2_Edge = combine_zone[0][1]

No_PV = (PV_Upper_Area_2_Edge - PV_Lower_Area_2_Edge + pv_gap)/(pv_width+pv_gap)
No_PV = floor(No_PV)
No_PV 

In [ ]:
#Calculate PV column in new area
st_x_Ref = combine_zone[1][0] + building_edge #1st Edge of PV Panel
x_nth = combine_zone[0][3] - building_edge  #Area 2 Edge for compare
g = 0
g1 = 0
w = 0
w1 = 0
for r in range(1,PV_set):
    
    if r > 2:
        
        if g1 > 1:
            g+=1
            
        if g1 == 2:
            g1 = 0
            
        g1 += 1
        
    if r == 2:
        g  += 1
        g1 += 1
        
    
        
    if r > No_column_set+1:
        
        if w1 > No_column_set-1:
            w += 1
        
        if w1 == No_column_set:
            w1 = 0
            
        w1 += 1
        
    if r == No_column_set+1:
        w += 1
        w1 += 1

    Ref_PV_edge_compare = st_x_Ref + r*pv_length + g*pv_gap + w*(walkway_width + 2*walkway_pv_gap)
    #print("r : "+str(r)+" w : "+str(w)+" w1 : "+str(w1))
    #print("r : "+str(r)+" g : "+str(g)+" w : "+str(w))
    
    if Ref_PV_edge_compare > x_nth:
        pv = r-1
        break

pv

In [ ]:
combine_zone[1][0] + building_edge, Lower_Main_Way

In [ ]:
#P2 = Panel.Copy()
PV_set = pv
no_B = 0
g = 0
x = combine_zone[1][0] + building_edge  #1st Edge of PV Panel
y = combine_zone[1][7] - building_edge - walkway_width - 3*walkway_endclamp #Back to start point
yi = Lower_Main_Way
Round = 0

# create empty list to store value of edge of x and y to find contact point of DC cables
x_edge_list_1 = []
y_edge_list_1 = []

num_pv_each_column_list = []
for c in range(0,PV_set):
    count_PV = 1
    
    if (c>0) & (c%No_column_set==0):
        no_B += 1
        
    if (c>0) & (Round>No_column_set):
        Round = 1
    if (c>0) & (Round<No_column_set):
        g +=1
        
    
    print("c : "+str(c)+" Round : "+str(Round)+" g : "+str(g))
    Round += 1
         
    xi = x+(c*pv_length)+(g*pv_gap)+no_B*(2*walkway_pv_gap+walkway_width)    
    P2 = Panel.Copy()
    P2.Move(APoint(x,y),
            APoint(xi,yi)) #Plus in "C" from "left" to "Right"

    for r in range(1,No_PV):
        
        y_new = yi-pv_width*r-pv_gap*r
        P2 = Panel.Copy()
        P2.Move(APoint(x,y),
            APoint(xi,y_new)) #Minus in "r" from "Up" to "Down"
        count_PV += 1
    x_edge = xi 
    y_edge = yi
    x_edge_list_1.append(x_edge)
    y_edge_list_1.append(y_edge)      
    #if (c>0) & (c%No_column_set==0):
    #    no_B += 1
    
    num_pv_each_column_list.append(count_PV)
    


In [ ]:
x_contact_point_list_1 = []
y_contact_point_list_1 = []
if len(x_edge_list_1)%2 == 0:
    for i in range (0,len(x_edge_list_1),2):
        x1 = x_edge_list_1[i]
        x2 = x_edge_list_1[i+1]+pv_length
        y1 = y_edge_list_1[i]
        y2 = y_edge_list_1[i+1]
        x_contact_point_list_1.append(x1)
        x_contact_point_list_1.append(x2)
        y_contact_point_list_1.append(y1)
        y_contact_point_list_1.append(y2)
        
elif len(x_edge_list_1)%2 != 0:
    for i in range (0,len(x_edge_list_1)-1,2):
        x1 = x_edge_list_1[i]
        x2 = x_edge_list_1[i+1]+pv_length
        y1 = y_edge_list_1[i]
        y2 = y_edge_list_1[i+1]
        x_contact_point_list_1.append(x1)
        x_contact_point_list_1.append(x2)
        y_contact_point_list_1.append(y1)
        y_contact_point_list_1.append(y2)
    x1 = x_edge_list_1[-1]
    x2 = x_edge_list_1[-1]+pv_length
    x_contact_point_list_1.append(x1)
    x_contact_point_list_1.append(x2)
    y = y_edge_list_1[-1]
    y_contact_point_list_1.append(y)
    y_contact_point_list_1.append(y)

# Create DC Cables

In [ ]:
# create dc line from contact point to inverter

y_center_main_way = (Main_way[1]+Main_way[7])/2 # y coordinate of center of main way
# create empty list to store length of dc line of each PV string
dc_length_list = []
for i in range (len(x_contact_point_list_1)):
    x1 = x_contact_point_list_1[i]
    y1 = y_contact_point_list_1[i]
    p1 = APoint(x1,y1)
        
    x2 = x1
    y2 = y_center_main_way
    p2 = APoint(x2,y2)
        
    x3 = inverter_coor[0]
    y3 = y_center_main_way
    p3 = APoint(x3,y3)
        
    x4 = x3
    y4 = inverter_coor[1]
    p4 = APoint(x4,y4)

    if i%2 == 0:
        line1 = acad.model.AddLine(p1,p2)
        line1.Color = 1
        line2 = acad.model.AddLine(p2,p3)
        line2.Color = 1
        line3 = acad.model.AddLine(p3,p4)
        line3.Color = 1
    elif i%2 != 0:
        line1 = acad.model.AddLine(p1,p2)
        line1.Color = 3
        line2 = acad.model.AddLine(p2,p3)
        line2.Color = 3
        line3 = acad.model.AddLine(p3,p4)
        line3.Color = 3
    
    length_part_1 = distance(p1,p2)
    length_part_2 = distance(p2,p3)
    length_part_3 = distance(p3,p4)
    
    dc_length = length_part_1 + length_part_2 + length_part_3 # length of dc line of each PV string
    dc_length_list.append(dc_length)

In [ ]:
dc_length_list

In [ ]:
num_pv_per_string_list = []
if len(num_pv_each_column_list)%No_column_set == 0:
    for i in range (0,len(num_pv_each_column_list)):
        num_PV = num_pv_each_column_list[i] * No_column_set
        num_pv_per_string_list.append(num_PV)
        
elif len(num_pv_each_column_list )%No_column_set != 0:
    for i in range (0,len(num_pv_each_column_list)-1):
        num_PV = num_pv_each_column_list[i] * No_column_set
        num_pv_per_string_list.append(num_PV)
    num_PV = num_pv_each_column_list [-1] # last num
    num_pv_per_string_list.append(num_PV)
    num_pv_per_string_list.append(num_PV)

In [ ]:
num_pv_per_string_list

In [ ]:
Iwire = 14.47 #unit: A
Rwire = 5.09
voltage_drop_list = [] # create empty list to store voltage drop of each PV string
for l in dc_length_list:
    r = 2*l*Rwire/1000 #unit: ohm
    voltage_drop = Iwire * r
    voltage_drop_list.append(voltage_drop)

In [ ]:
# create pandas datafrane where colums are voltage drop, length of dc line, and number of PV
DC_cable_dataframe_1 = pd.DataFrame()
DC_cable_dataframe_1['length (m)'] = dc_length_list
DC_cable_dataframe_1['num_PV_per_string'] = num_pv_per_string_list
DC_cable_dataframe_1['voltage_at_start_terminal (V)'] = DC_cable_dataframe_1['num_PV_per_string'] * 51.02

DC_cable_dataframe_1['voltage_drop (V)'] = voltage_drop_list
DC_cable_dataframe_1['percent_voltage_drop (%)'] = DC_cable_dataframe_1['voltage_drop (V)'] / DC_cable_dataframe_1['voltage_at_start_terminal (V)'] * 100


for percent in DC_cable_dataframe_1['percent_voltage_drop (%)']:
    if percent < 3:
        DC_cable_dataframe_1['DC_cable_size (mm^2)'] = 4
    elif 3 < percent and percent < 6:
        DC_cable_dataframe_1['DC_cable_size (mm^2)'] = 6
    elif 6 < percent and percent < 9:
        DC_cable_dataframe_1['DC_cable_size (mm^2)'] = 10

for size in DC_cable_dataframe_1['DC_cable_size (mm^2)']:
    if size == 4:
        DC_cable_dataframe_1['DC_cable_price (THB)'] = DC_cable_dataframe_1['length (m)'] * 20
    elif size == 6:
        DC_cable_dataframe_1['DC_cable_price (THB)'] = DC_cable_dataframe_1['length (m)'] * 25
    elif size == 10:
        DC_cable_dataframe_1['DC_cable_price (THB)'] = DC_cable_dataframe_1['length (m)'] * 45
        
DC_cable_dataframe_1

In [ ]:
total_DC_cable_price_1 = DC_cable_dataframe_1['DC_cable_price (THB)'].sum()
print(f'Total Price is {total_DC_cable_price_1} Baht')

In [ ]:
total_DC_cost = total_DC_cable_price + total_DC_cable_price_1
print(f'Total DC cost is {total_DC_cost} Baht')

In [ ]:
combine_zone[0][1] + walkway_width

# Create Branch

In [ ]:
#Walk Way create (Branch)
branch_way = []
x = combine_zone[1][0] + building_edge  #start point
y = yi + walkway_endclamp  #start point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x + walkway_width  #right point
y = y #right point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x #right bottom point
y = combine_zone[0][1] + walkway_width - 3*walkway_endclamp  #right bottom point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = x - walkway_width #right bottom point
y = y  #right bottom point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

x = combine_zone[1][0] + building_edge  #start point
y = yi + walkway_endclamp #start point
z = 0
edge = [x,y,z]
branch_way.extend(edge)

bw = aDouble(branch_way)
BranchWay = acad.model.AddPolyline(bw)
BranchWay.Color = 2

In [ ]:
#Expand branch line

Pre_Walk = No_column_set*pv_length+(No_column_set-1)*pv_gap+walkway_pv_gap
Back_Walk = walkway_pv_gap + walkway_width
x = combine_zone[1][0] + building_edge  #start point
y = combine_zone[1][7] - building_edge - walkway_width - 0*walkway_endclamp #start point
Round = 0
pv = 0
m_point = Pre_Walk*(Round+1) + Round*Back_Walk
BranchWay.Move(APoint(x,y),
            APoint(x+m_point,y)) #Plus in "C" from "left" to "Right"
x = x+m_point

#bw2 = BranchWay.Copy()
if PV_set%No_column_set!=0:
    PV_set_Round = PV_set-1
if PV_set%No_column_set==0:
    PV_set_Round = PV_set
    
while pv < PV_set_Round:
    #print("pv : "+str(pv))
    bw2 = BranchWay.Copy()
    m_point = (2*walkway_pv_gap + walkway_width + No_column_set*pv_length + (No_column_set-1)*pv_gap)*Round
    #m_point = Pre_Walk*(Round+1) + Round*Back_Walk
    pv = pv + No_column_set
    Round += 1
    
    bw2.Move(APoint(x,y),
            APoint(x+m_point,y)) #Plus in "C" from "left" to "Right"

# Total Cost

In [ ]:
DC_cable_dataframe

In [ ]:
DC_cable_dataframe_1

In [ ]:
print(f'Total cost is {total_DC_cost.round(2)} Baht')